Project: /mediapipe/_project.yaml
Book: /mediapipe/_book.yaml

<link rel="stylesheet" href="/mediapipe/site.css">

# Object detection model customization guide

<table align="left" class="buttons">
  <td>
    <a href="https://colab.research.google.com/github/googlesamples/mediapipe/blob/main/examples/customization/object_detector.ipynb" target="_blank">
      <img src="https://developers.google.com/static/mediapipe/solutions/customization/colab-logo-32px_1920.png" alt="Colab logo"> Run in Colab
    </a>
  </td>

  <td>
    <a href="https://github.com/googlesamples/mediapipe/blob/main/examples/customization/object_detector.ipynb" target="_blank">
      <img src="https://developers.google.com/static/mediapipe/solutions/customization/github-logo-32px_1920.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

In [ ]:
#@title License information
# Copyright 2023 The MediaPipe Authors.
# Licensed under the Apache License, Version 2.0 (the "License");
#
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

The MediaPipe object detection solution provides several models you can use immediately for machine learning (ML) in your application. However, if you need to detect objects not covered by the provided models, you can customize any of the provided models with your own data and MediaPipe Model Maker. This model modification tool rebuilds the model using data you provide. This method is faster than training a new model and can produce a model that is more useful for your specific application.

The following sections show you how to use Model Maker to retrain a pre-built model for object detection with your own data, which you can then use with the MediaPipe [Object Detector](https://developers.google.com/mediapipe/solutions/vision/object_detector). The example retrains a general purpose object detection model to detect android figurines in images.

## Setup

This section describes key steps for setting up your development environment to retrain a model. These instructions describe how to update a model using [Google Colab](https://colab.research.google.com/), and you can also use Python in your own development environment. For general information on setting up your development environment for using MediaPipe, including platform version requirements, see the [Setup guide for Python](https://developers.google.com/mediapipe/solutions/setup_python).

**Attention:** This MediaPipe Solutions Preview is an early release. [Learn more](https://developers.google.com/mediapipe/solutions/about).

To install the libraries for customizing a model, run the following commands:

In [ ]:
!python --version
!pip install --upgrade pip
!pip install mediapipe-model-maker
!pip install roboflow  # 需要用其下载数据集
!pip install comet_ml  # 需要用其记录训练数据

Use the following code to import the required Python classes:

In [ ]:
from google.colab import files
import os
import json
import tensorflow as tf
assert tf.__version__.startswith('2')
from mediapipe_model_maker import object_detector



实验数据记录

In [ ]:
from comet_ml import start
from comet_ml.integration.pytorch import log_model

experiment = start(
  api_key="L9Hl7JDyQloSScOBXqjixAq09",
  project_name="volley-detect",
  workspace="liyunkai13"
)
hyper_params = {
   "learning_rate": 0.5,
   "steps": 100000,
   "batch_size": 50,
}
experiment.log_parameters(hyper_params)

## Prepare data

Retraining a model for object detection requires a dataset that includes the items, or classes, that you want the completed model to be able to identify. You can do this by trimming down a public dataset to only the classes that are relevant to your usecase, compiling your own dataset, or some combination of both, The dataset can be significantly smaller than what would be required to train a new model. For example, the [COCO](https://cocodataset.org/) dataset used to train many reference models contains hundreds of thousands of images with 91 classes of objects. Transfer learning with Model Maker can retrain an existing model with a smaller dataset and still perform well, depending on your inference accuracy goals. These instructions use a smaller dataset containing 2 types of android figurines, or 2 classes, with 62 total training images.

To download the example dataset, use the following code:

### 从Roboflow下载数据集

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="83pzRFncTtpun9yDYi5f")
project = rf.workspace("volleyball-mrfzh").project("volley_detect")
version = project.version(1)
dataset = version.download("coco")


### 从谷歌云盘获取数据集

In [ ]:
# 把谷歌云盘挂载在虚拟机上
from google.colab import drive
drive.mount('/content/drive')
# 把数据局文件复制到本地目录
!cp "/content/drive/MyDrive/Volleyball.v1i.coco.zip" "/content/"

### 若为压缩包，还需要解压

In [ ]:
# 解压数据集
import zipfile

with zipfile.ZipFile('/content/Volleyball.v1i.coco.zip', 'r') as zip_ref:
  zip_ref.extractall('/content/dataset')

This code stores the dataset at the directory location `android_figurine`. The directory contains two subdirectories for the training and validation datasets, located in `android_figurine/train` and `android_figurine/validation` respectively. Each of the train and validation datasets follow the COCO Dataset format described below.

### 支持的数据集格式
Model Maker Object Detection API 支持读取这样的数据集格式:

#### COCO format

COCO 格式的数据集有一个 data 文件夹，其包含所有的图片和一个  `labels.json` 文件，这个文件包含着所有图片的对象标注
```
<dataset_dir>/
  data/
    <img0>.<jpg/jpeg>
    <img1>.<jpg/jpeg>
    ...
  labels.json
```
where `labels.json` is formatted as:
```
{
  "categories":[
    {"id":1, "name":<volleyball>},
    ...
  ],
  "images":[
    {"id":0, "file_name":"<img0>.<jpg/jpeg>"},
    ...
  ],
  "annotations":[
    {"id":0, "image_id":0, "category_id":1, "bbox":[x-top left, y-top left, width, height]},
    ...
  ]
}
```

#### PASCAL VOC format

The PASCAL VOC dataset format also has a `data` directory which stores all of the images, however the annotations are split up per image into corresponding xml files in the `Annotations` directory.
```
<dataset_dir>/
  data/
    <file0>.<jpg/jpeg>
    ...
  Annotations/
    <file0>.xml
    ...
```
where the xml files are formatted as:
```
<annotation>
  <filename>file0.jpg</filename>
  <object>
    <name>kangaroo</name>
    <bndbox>
      <xmin>233</xmin>
      <ymin>89</ymin>
      <xmax>386</xmax>
      <ymax>262</ymax>
    </bndbox>
  </object>
  <object>
    ...
  </object>
  ...
</annotation>
```

### 数据集预览

从json文件中打印类别来验证数据集的内容。

In [ ]:
with open(os.path.join("/content/volley_detect-1/train","_annotations.coco.json"), "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")

为了更好地理解数据集，绘制两个示例图像及其边框

In [ ]:
# @title Visualize the training dataset {"vertical-output":true}
variable_name = "" # @param {"type":"string"}
variable_name = "" # @param {"type":"raw"}
import matplotlib.pyplot as plt
from matplotlib import patches, text, patheffects
from collections import defaultdict
import math

def draw_outline(obj):
  obj.set_path_effects([patheffects.Stroke(linewidth=4,  foreground='black'), patheffects.Normal()])
def draw_box(ax, bb):
  patch = ax.add_patch(patches.Rectangle((bb[0],bb[1]), bb[2], bb[3], fill=False, edgecolor='red', lw=2))
  draw_outline(patch)
def draw_text(ax, bb, txt, disp):
  text = ax.text(bb[0],(bb[1]-disp),txt,verticalalignment='top'
  ,color='white',fontsize=10,weight='bold')
  draw_outline(text)
def draw_bbox(ax, annotations_list, id_to_label, image_shape):
  for annotation in annotations_list:
    cat_id = annotation["category_id"]
    bbox = annotation["bbox"]
    draw_box(ax, bbox)
    draw_text(ax, bbox, id_to_label[cat_id], image_shape[0] * 0.05)
def visualize(dataset_folder, max_examples=None):
  with open(os.path.join(dataset_folder,"train","_annotations.coco.json"), "r") as f:
    labels_json = json.load(f)
  images = labels_json["images"]
  cat_id_to_label = {item["id"]:item["name"] for item in labels_json["categories"]}
  image_annots = defaultdict(list)
  for annotation_obj in labels_json["annotations"]:
    image_id = annotation_obj["image_id"]
    image_annots[image_id].append(annotation_obj)

  if max_examples is None:
    max_examples = len(image_annots.items())
  n_rows = math.ceil(max_examples / 3)
  fig, axs = plt.subplots(n_rows, 3, figsize=(24, n_rows*8)) # 3 columns(2nd index), 8x8 for each image
  for ind, (image_id, annotations_list) in enumerate(list(image_annots.items())[:max_examples]):
    ax = axs[ind//3, ind%3]
    img = plt.imread(os.path.join(dataset_folder, "train", images[image_id]["file_name"]))
    ax.imshow(img)
    draw_bbox(ax, annotations_list, cat_id_to_label, img.shape)
  plt.show()

# visualize(train_dataset_path, 9)
visualize("/content/volley_detect-1", 9)

### 创建数据集

数据集类有从COCO和PASCAL VOC数据集中加载的两种方法:
* `Dataset.from_coco_folder`
* `Dataset.from_pascal_voc_folder`

由于所用数据集是COCO格式，使用 `from_coco_folder`方法来加载位于 `train_dataset_path` and `validation_dataset_path`两个路径下的数据集。加载数据集时，数据会被转换成标准的 [TFRecord](https://www.tensorflow.org/tutorials/load_data/tfrecord)格式，被缓存作之后使用。应该创建一个 `cache_dir`文件夹并复用其在每一次训练中，避免存储同一个数据集的多个缓存。

#### 数据集组织格式处理

In [ ]:
# 因为数据集内格式与from_coco_folder方法的预想格式一致，写个脚本把train下边的所有图片
# 放在train/images中

import os
import shutil

def move_images_to_subfolder(data_dir, subfolder_name="images"):
  """将指定目录下的所有图像文件移动到名为 subfolder_name 的子目录中。

  Args:
    data_dir: 数据集所在的目录路径。
    subfolder_name: 子目录的名称，默认为 "images"。
  """

  # 创建子目录
  subfolder_path = os.path.join(data_dir, subfolder_name)
  os.makedirs(subfolder_path, exist_ok=True)

  # 遍历数据目录下的所有文件
  for filename in os.listdir(data_dir):
    # 检查文件是否为图像文件
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
      # 获取文件的完整路径
      source_path = os.path.join(data_dir, filename)
      # 获取目标路径
      destination_path = os.path.join(subfolder_path, filename)
      # 移动文件
      shutil.move(source_path, destination_path)

# 调用函数，将 /content/dataset/train 目录下的图像移动到 images 子目录中
move_images_to_subfolder("/content/volley_detect-1/train")
move_images_to_subfolder("/content/volley_detect-1/valid")
move_images_to_subfolder("/content/volley_detect-1/test")

print("图像文件已移动到 images 子目录中。")



# 这个傻逼函数还预设json文件名为labels.json，只能改名

由于该数据集只有两个类，都是volleyball，没有背景类，而modelmaker要求有一个背景类并且编号为0，故需要做额外处理

In [ ]:
import json
import os

def add_background_class_to_coco_json(json_file_path):
  """向 COCO JSON 文件添加背景类，并更新现有类别的 ID。

  Args:
      json_file_path: COCO JSON 文件的路径。
  """
  with open(json_file_path, 'r') as f:
    data = json.load(f)

  # 添加背景类
  data['categories'].insert(0, {'id': 0, 'name': 'background'})

  # 更新现有类别的 ID
  for category in data['categories'][1:]:  # 从索引 1 开始，跳过背景类
    category['id'] += 1

  # 更新注释中的类别 ID
  for annotation in data['annotations']:
    annotation['category_id'] += 1  # 所有类别 ID 递增 1

  with open(json_file_path, 'w') as f:
    json.dump(data, f, indent=2)

  print(f"已向 {json_file_path} 添加背景类，并更新了现有类别的 ID。")

# 调用函数
add_background_class_to_coco_json("/content/volley_detect-1/train/labels.json")
add_background_class_to_coco_json("/content/volley_detect-1/valid/labels.json")
add_background_class_to_coco_json("/content/volley_detect-1/test/labels.json")

In [ ]:
train_data = object_detector.Dataset.from_coco_folder(
    "/content/volley_detect-1/train",
    cache_dir="/tmp/od_data/train"
    )
validation_data = object_detector.Dataset.from_coco_folder("/content/volley_detect-1/valid", cache_dir="/tmp/od_data/validation")
test_data = object_detector.Dataset.from_coco_folder("/content/volley_detect-1/test", cache_dir="/tmp/od_data/test")
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)
print("test_data size: ", test_data.size)

## 若无测试集和验证集
存在数据集只有训练集的情况，而重新训练要求必须有验证集，在创建模型时，不传入 validation_data 参数，模型将不会进行验证。有几种处理方法

1. 将训练集拆分为训练集和验证集

In [ ]:
from sklearn.model_selection import train_test_split

# # 将 Dataset 对象转换为列表，才能使用train_test_split函数
# data_list = list(train_data)

# # 拆分列表
# train_data_list, validation_data_list = train_test_split(data_list, test_size=0.2, random_state=42)

# # 将拆分后的列表转换回 Dataset 对象
# train_data = object_detector.Dataset(train_data_list)
# validation_data = object_detector.Dataset(validation_data_list)

# # 将 train_data 拆分为训练集和验证集
# train_data, validation_data = train_test_split(train_data, test_size=0.2, random_state=42)  # test_size 表示验证集的比例



train_data, validation_data = train_data.split(0.8)  # 80% 用于训练，20% 用于验证

print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)


2. 使用 k 折交叉验证  
k 折交叉验证是一种更 robust 的评估模型性能的方法，它将训练集分成 k 个子集，每次使用 k-1 个子集进行训练，剩余的 1 个子集作为验证集。重复 k 次，每次使用不同的子集作为验证集，最终的性能指标是 k 次验证结果的平均值。

In [ ]:
# from sklearn.model_selection import KFold

# # 创建 k 折交叉验证器
# kf = KFold(n_splits=5, shuffle=True, random_state=42)  # n_splits 表示将数据集分成 5 个子集

# # 循环进行训练和评估
# for train_index, val_index in kf.split(train_data):
#   train_data_fold = train_data[train_index]  # 当前 fold 的训练集
#   validation_data_fold = train_data[val_index]  # 当前 fold 的验证集

#   # 创建模型
#   model = object_detector.ObjectDetector.create(
#       train_data=train_data_fold,
#       validation_data=validation_data_fold,  # 传入当前 fold 的验证集
#       options=options
#   )


## 重新训练模型

数据准备结束后, 可以开始重新训练模型来识别根据训练数据新定义的目标或者类。以下内容介绍使用之前准备的数据来训练模型识别新类。

### 重新训练设置项
除了训练数据集之外，还有一些运行再训练所需的设置： 模型的输出目录和模型架构。使用 HParams 为输出目录指定`export_dir`参数。使用 SupportedModel 类指定模型架构。mediapipe对象检测器解决方案支持以下模型架构：
* `MobileNet-V2`
* `MobileNet-MultiHW-AVG`

更多高级的训练参数自定义设置，看[Hyperparameters](#hyperparameters) section below.

设置必要参数，使用以下代码：

In [ ]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(export_dir='/content/exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

### 运行 重新训练
With your training dataset and retraining options prepared, you are ready to start the retraining process. This process is resource intensive and can take a few minutes to a few hours depending on your available compute resources. Using a Google Colab environment with standard GPU runtimes, the example retraining below takes about 2~4 minutes.

To begin the retraining process, use the `create()` method with dataset and options you previously defined:

In [ ]:
# 创建模型
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,  # 传入验证集
    options=options
)


### 评估模型表现

在训练模型之后，在验证集上评估并打印出loss和coco_metrics,评估模型表现最重要的度量是 typically the "AP" coco metric for Average Precision.

上传结果到comet

In [ ]:
log_model(experiment, model=model, model_name="mediapipe_detect")

本地评估下

In [ ]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

## Export model

After creating the model, convert and export it to a Tensorflow Lite model format for later use on an on-device application. The export also includes model metadata, which includes the label map.

In [ ]:
model.export_model()
!ls exported_model
files.download('exported_model/mediapipe_detect.tflite')

## 模型量化

Model quantization is a model modification technique that can reduce the model size and improve the speed of predictions with only a relatively minor decrease in accuracy.

This section of the guide explains how to apply quantization to your model. Model Maker supports two forms of quantization for object detector:
1. Quantization Aware Training: 8 bit integer precision for CPU usage
2. Post-Training Quantization: 16 bit floating point precision for GPU usage

### Quantization aware training (int8 quantization)
Quantization aware training (QAT) is a fine-tuning step which happens after fully training your model. This technique further tunes a model which emulates inference time quantization in order to account for the lower precision of 8 bit integer quantization. For on-device applications with a standard CPU, use Int8 precision. For more information, see the [TensorFlow Lite](https://www.tensorflow.org/model_optimization/guide/quantization/training) documentation.

To apply quantization aware training and export to an int8 model, create a `QATHParams` configuration and run the `quantization_aware_training` method. See the **Hyperparameters** section below on detailed usage of `QATHParams`.

In [ ]:
qat_hparams = object_detector.QATHParams(learning_rate=0.3, batch_size=4, epochs=10, decay_steps=6, decay_rate=0.96)
model.quantization_aware_training(train_data, validation_data, qat_hparams=qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")

The QAT step often requires multiple runs to tune the parameters of training. To avoid having to rerun model training using the `create` method, use the `restore_float_ckpt` method to restore the model state back to the fully trained float model(After running the `create` method) in order to run QAT again.

In [ ]:
new_qat_hparams = object_detector.QATHParams(learning_rate=0.9, batch_size=4, epochs=15, decay_steps=5, decay_rate=0.96)
model.restore_float_ckpt()
model.quantization_aware_training(train_data, validation_data, qat_hparams=new_qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")

Finally, us the `export_model` to export to an int8 quantized model. The `export_model` function will automatically export to either float32 or int8 model depending on whether `quantization_aware_training` was run.

In [ ]:
model.export_model('model_int8_qat.tflite')
!ls -lh exported_model
files.download('exported_model/model_int8_qat.tflite')

### Post-training quantization (fp16 quantization)

Post-training model quantization is a model modification technique that can reduce the model size and improve the speed of predictions with only a relatively minor decrease in accuracy. This approach reduces the size of the data processed by the model, for example by transforming 32-bit floating point numbers to 16-bit floats. Float16 quantization is reccomended for GPU usage. For more information, see the [TensorFlow Lite](https://www.tensorflow.org/model_optimization/guide/quantization/post_training) documentation.

First, import the MediaPipe Model Maker quantization module:

In [ ]:
from mediapipe_model_maker import quantization

Define a QuantizationConfig object using the `for_float16()` class method. This configuration modifies a trained model to use 16-bit floating point numbers instead of 32-bit floating point numbers. You can further customize the quantization process by setting additional parameters for the QuantizationConfig class.

In [ ]:
quantization_config = quantization.QuantizationConfig.for_float16()

Export the model using the additional quantization_config object to apply post-training quantization. Note that if you previously ran `quantization_aware_training`, you must first convert the model back to a float model by using `restore_float_ckpt`.

In [ ]:
model.restore_float_ckpt()
model.export_model(model_name="model_fp16.tflite", quantization_config=quantization_config)
!ls -lh exported_model
files.download('exported_model/model_fp16.tflite')

## Hyperparameters
You can further customize the model using the ObjectDetectorOptions class, which has three parameters for `SupportedModels`, `ModelOptions`, and `HParams`.

Use the `SupportedModels` enum class to specify the model architecture to use for training. The following model architectures are supported:
* MOBILENET_V2
* MOBILENET_V2_I320
* MOBILENET_MULTI_AVG
* MOBILENET_MULTI_AVG_I384

Use the `HParams` class to customize other parameters related to training and saving the model:
* `learning_rate`: Learning rate to use for gradient descent training. Defaults to 0.3.
* `batch_size`: Batch size for training. Defaults to 8.
* `epochs`: Number of training iterations over the dataset. Defaults to 30.
* `cosine_decay_epochs`: The number of epochs for cosine decay learning rate. See [tf.keras.optimizers.schedules.CosineDecay](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/CosineDecay) for more info. Defaults to None, which is equivalent to setting it to `epochs`.
* `cosine_decay_alpha`: The alpha value for cosine decay learning rate. See [tf.keras.optimizers.schedules.CosineDecay](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/CosineDecay) for more info. Defaults to 1.0, which means no cosine decay.

Use the `ModelOptions` class to customize parameters related to the model itself:
* `l2_weight_decay`: L2 regularization penalty used in [tf.keras.regularizers.L2](https://www.tensorflow.org/api_docs/python/tf/keras/regularizers/L2). Defaults to 3e-5.

Uset the `QATHParams` class to customize training parameters for Quantization Aware Training:
* `learning_rate`: Learning rate to use for gradient descent QAT. Defaults to 0.3.
* `batch_size`: Batch size for QAT. Defaults to 8
* `epochs`: Number of training iterations over the dataset. Defaults to 15.
* `decay_steps`: Learning rate decay steps for Exponential Decay. See [tf.keras.optimizers.schedules.ExponentialDecay](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/ExponentialDecay) for more information. Defaults to 8
* `decay_rate`: Learning rate decay rate for Exponential Decay. See [tf.keras.optimizers.schedules.ExponentialDecay](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/ExponentialDecay) for more information. Defaults to 0.96.

## Benchmarking
Below is a summary of our benchmarking results for the supported model architectures. These models were trained and evaluated on the same android figurines dataset as this notebook. When considering the model benchmarking results, there are a few important caveats to keep in mind:
* The android figurines dataset is a small and simple dataset with 62 training examples and 10 validation examples. Since the dataset is quite small, metrics may vary drastically due to variances in the training process. This dataset was provided for demo purposes and it is recommended to collect more data samples for better performing models.
* The float32 models were trained with the default HParams, and the QAT step for the int8 models was run with `QATHParams(learning_rate=0.1, batch_size=4, epochs=30, decay_rate=1)`.
* For your own dataset, you will likely need to tune values for both HParams and QATHParams in order to achieve the best results. See the [Hyperparameters](#hyperparameters) section above for more information on configuring training parameters.
* All latency numbers are benchmarked on the Pixel 6.


<table>
<thead>
<col>
<col>
<colgroup span="2"></colgroup>
<colgroup span="2"></colgroup>
<colgroup span="2"></colgroup>
<tr>
<th rowspan="2">Model architecture</th>
<th rowspan="2">Input Image Size</th>
<th colspan="2" scope="colgroup">Test AP</th>
<th colspan="2" scope="colgroup">CPU Latency</th>
<th colspan="2" scope="colgroup">Model Size</th>
</tr>
<tr>
<th>float32</th>
<th>QAT int8</th>
<th>float32</th>
<th>QAT int8</th>
<th>float32</th>
<th>QAT int8</th>
</tr>
</thead>
<tbody>
<tr>
<td>MobileNetV2</td>
<td>256x256</td>
<td>88.4%</td>
<td>73.5%</td>
<td>48ms</td>
<td>16ms</td>
<td>11MB</td>
<td>3.2MB</td>
</tr>
<tr>
<td>MobileNetV2 I320</td>
<td>320x320</td>
<td>89.1%</td>
<td>75.5%</td>
<td>75ms</td>
<td>33.38ms</td>
<td>10MB</td>
<td>3.3MB</td>
</tr>
<tr>
<td>MobileNet MultiHW AVG</td>
<td>256x256</td>
<td>88.5%</td>
<td>70.0%</td>
<td>56ms</td>
<td>19ms</td>
<td>13MB</td>
<td>3.6MB</td>
</tr>
<tr>
<td>MobileNet MultiHW AVG I384</td>
<td>384x384</td>
<td>92.7%</td>
<td>73.4%</td>
<td>238ms</td>
<td>41ms</td>
<td>13MB</td>
<td>3.6MB</td>
</tr>

</tbody>
</table>

